In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

structure = '10N_R'
atlas_path = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/atlasV8'
volume_path = os.path.join(atlas_path, 'structure', f'{structure}.npy')
volume = np.load(volume_path)

volume.shape

In [ ]:
%%time
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

cmap = plt.get_cmap("viridis")
norm= plt.Normalize(volume.min(), volume.max())
ax.voxels(np.ones_like(volume), facecolors=cmap(norm(volume)), edgecolor="black")
#ax.voxels(volume)
plt.show()